# Running ProjectQ code on AWS Braket service provided devices
## Compiling code for AWS Braket Service

In this tutorial we will see how to run code on some of the devices provided by the Amazon AWS Braket service. The AWS Braket devices supported are: the State Vector Simulator 'SV1', the Rigetti device 'Aspen-8' and the IonQ device 'IonQ'

You need to have a valid AWS account, created a pair of access key/secret key, and have activated the braket service. As part of the activation of the service, a specific S3 bucket and folder associated to the service should be configured.

First we need to do the required imports. That includes the mail compiler engine (MainEngine), the backend (AWSBraketBackend in this case) and the operations to be used in the cicuit

In [ ]:
from projectq import MainEngine
from projectq.backends import AWSBraketBackend
from projectq.ops import Measure, H, C, X, All


Prior to the instantiation of the backend we need to configure the credentials, the S3 storage folder and the device to be used (in the example the State Vector Simulator SV1)

In [ ]:
creds = {
    'AWS_ACCESS_KEY_ID': 'aws_access_key_id',
    'AWS_SECRET_KEY': 'aws_secret_key',
    } # replace with your Access key and Secret key

s3_folder = ['S3Bucket', 'S3Directory'] # replace with your S3 bucket and directory

device = 'SV1' # replace by the device you want to use

Next we instantiate the engine with the AWSBraketBackend including the credentials and S3 configuration. By setting the 'use_hardware' parameter to False we indicate the use of the Simulator. In addition we set the number of times we want to run the circuit and the interval in secons to ask for the results. For a complete list of parameters and descriptions, please check the documentation.

In [ ]:
eng = MainEngine(AWSBraketBackend(use_hardware=False,
                                  credentials=creds,
                                  s3_folder=s3_folder,
                                  num_runs=10,
                                  interval=10))

We can now allocate the required qubits and create the circuit to be run. With the last instruction we ask the backend to run the circuit.

In [ ]:
# Allocate the required qubits
qureg = eng.allocate_qureg(3)

# Create the circuit. In this example a quantum teleportation algorithms that teleports the first qubit to the third one.
H | qureg[0]
H | qureg[1]
C(X) | (qureg[1], qureg[2])
C(X) | (qureg[0], qureg[1])
H | qureg[0]
C(X) | (qureg[1], qureg[2])

# At the end we measure the qubits to get the results; should be all-0 or all-1
All(Measure) | qureg

# And run the circuit
eng.flush()


The backend will automatically create the task and generate a unique identifier (the task Arn) that can be used to recover the status of the task and results later on.

Once the circuit is executed the indicated number of times, the results are stored in the S3 folder configured previously and can be recovered to obtain the probabilities of each of the states.

In [ ]:
# Obtain and print the probabilies of the states
prob_dict = eng.backend.get_probabilities(qureg)
print("Probabilites for each of the results: ", prob_dict)

## Retrieve results form a previous execution

We can retrieve the result later on (of this job or a previously executed one) using the task Arn provided when it was run. In addition, you have to remember the amount of qubits involved in the job and the order you used. The latter is required since we need to set up a mapping for the qubits when retrieving results of a previously executed job.

To retrieve the results we need to configure the backend including the parameter 'retrieve_execution' set to the Task Arn of the job. To be able to get the probabilities of each state we need to configure the qubits and ask the backend to get the results.

In [ ]:
# Set the Task Arn of the job to be retrieved and instantiate the engine with the AWSBraketBackend
task_arn = 'your_task_arn' # replace with the actual TaskArn you want to use

eng1 = MainEngine(AWSBraketBackend(retrieve_execution=task_arn, credentials=creds, num_retries=2, verbose=True))

# Configure the qubits to get the states probabilies
qureg1 = eng1.allocate_qureg(3)

# Ask the backend to retrieve the results
eng1.flush()

# Obtain and print the probabilities of the states
prob_dict1 = eng1.backend.get_probabilities(qureg1)
print("Probabilities ", prob_dict1)


We can plot an histogram with the probabilities as well.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from projectq.libs.hist import histogram

histogram(eng1.backend, qureg1)
plt.show()